# file is present inside a folder called notebooks inside Project directory

In [169]:
import numpy as np
import random
from itertools import permutations

from sklearn.metrics import accuracy_score
from tqdm import tqdm
from scipy.spatial.distance import mahalanobis, jaccard, cosine

import sys
sys.path.append('../')

from utils import get_all_vectors, filter_images
from feature_reduction.feature_reduction import reducer
from metric.distance import distance

In [170]:
models = ['moment']
label = 'dorsal'
k = 20
frt = 'pca'

In [3]:
def generate_vec():
    
    #TRAIN DATA
    dorsal_paths = filter_images(label)
    dorsal_vectors, palmar_vectors = np.array([]), np.array([])
    for model in models:

        paths, temp = get_all_vectors(model, f={'path': {'$in': dorsal_paths}})
        if not dorsal_vectors.size: dorsal_vectors = temp
        else: dorsal_vectors = np.concatenate((dorsal_vectors, temp), axis=1)

        _, temp = get_all_vectors(model, f={'path': {'$nin': dorsal_paths}})
        if not palmar_vectors.size: palmar_vectors = temp
        else: palmar_vectors = np.concatenate((palmar_vectors, temp), axis=1)
    
#     dorsal_vectors, palmar_vectors = normalize(dorsal_vectors), normalize(palmar_vectors)
    
    #TEST DATA
    q_dorsal_paths = filter_images(label, unlabelled_db=True)
    q_dorsal_vectors, q_palmar_vectors = np.array([]), np.array([])
    for model in models:

        _, temp = get_all_vectors(model, f={'path': {'$in': q_dorsal_paths}}, unlabelled_db=True)
        if not q_dorsal_vectors.size: q_dorsal_vectors = temp
        else: q_dorsal_vectors = np.concatenate((q_dorsal_vectors, temp), axis=1)

        _, temp = get_all_vectors(model, f={'path': {'$nin': q_dorsal_paths}}, unlabelled_db=True)
        if not q_palmar_vectors.size: q_palmar_vectors = temp
        else: q_palmar_vectors = np.concatenate((q_palmar_vectors, temp), axis=1)

    q_dorsal_class = np.array([1] * len(q_dorsal_vectors))
    q_palmar_class = np.array([0] * len(q_palmar_vectors))
    
    test_data = np.vstack(
        (q_dorsal_vectors, q_palmar_vectors)
    )
    test_labels = np.concatenate((q_dorsal_class, q_palmar_class))
    
#     test_data = normalize(test_data)
    return dorsal_vectors, palmar_vectors, test_data, test_labels

In [17]:
results = []
done = []
for i in range(0,len(all_models) + 1):
    for subset in permutations(all_models, i):
        
        models = sorted(list(subset))
        if models in done: continue 
        if models:
            c_list = [5,10]
            k_list = [10, 20, 30]
            dorsal_vectors, palmar_vectors, test_data, test_labels = generate_vec()
            for k in k_list:
                for c in c_list:
                    print("Running ", c, k, models)
                    score = test(
                            dorsal_vectors, palmar_vectors, test_data, test_labels, c, k
                        )

                    res = {
                        'model': models,
                        'c': c,
                        'k': k,
                        'score': score
                    }
                    print(score)
                    results.append(res)

            done.append(models)

Running  5 10 ['moment']
negative found after scaling query vector. setting to 0
negative found after scaling query vector. setting to 0
0.34
Running  10 10 ['moment']


KeyboardInterrupt: 

In [387]:
models = ['sift']
dorsal_vectors, palmar_vectors, test_data, test_labels = generate_vec()

In [388]:
#PCA
pca_dorsal_vectors, _, _, dorsal_scaler, dorsal_pca = reducer( dorsal_vectors, k, frt, get_scaler_model = True)
pca_palmar_vectors, _, _, palmar_scaler, palmar_pca = reducer( palmar_vectors, k, frt, get_scaler_model = True)

In [389]:
dorsal_inv = np.linalg.inv(np.cov(pca_dorsal_vectors.T))
d_centroid = np.mean(pca_dorsal_vectors, axis=0)
d_centroid

array([ 2.66453526e-17,  5.49560397e-17, -8.88178420e-18,  5.10702591e-17,
        0.00000000e+00, -6.21943098e-17])

In [390]:
palmar_inv = np.linalg.inv(np.cov(pca_palmar_vectors.T))
p_centroid = np.mean(pca_palmar_vectors, axis=0)
p_centroid

array([ 8.88178420e-18,  1.33226763e-17,  7.32747196e-17, -5.32907052e-17,
        8.88178420e-18, -3.89772154e-17])

In [391]:
mahab_dorsal = []
for each in pca_dorsal_vectors:
    mahab_dorsal.append(mahalanobis(
        each, d_centroid, dorsal_inv
    ))
np.mean(mahab_dorsal)

2.299824560329968

In [392]:
mahab_palmar = []
for each in pca_palmar_vectors:
    mahab_palmar.append(mahalanobis(
        each, p_centroid, palmar_inv
    ))
np.mean(mahab_palmar)

2.253992515936915

In [393]:
i = 58
def test(i):
    d = mahalanobis( dorsal_pca.transform(dorsal_scaler.transform(test_data[i].reshape(1,-1)))
    , d_centroid, dorsal_inv)
    p = mahalanobis( palmar_pca.transform(palmar_scaler.transform(test_data[i].reshape(1,-1)))
    , p_centroid, palmar_inv)
    if d < p: return 1
    else: return 0

In [394]:
preds = [test(i) for i in range(len(test_data))]

In [395]:
accuracy_score(preds, test_labels)

0.64

# MAHAB

In [533]:
models = ['moment_inv']
frt = 'nmf'
k = 20
dorsal_vectors, palmar_vectors, test_data, test_labels = generate_vec()

In [534]:
pca_dorsal_vectors, _, _, q_pca_dorsal_vectors = reducer(
    dorsal_vectors,
    k,
    frt,
    query_vector = test_data
)

In [535]:
pca_dorsal_vectors = np.vstack((pca_dorsal_vectors, q_pca_dorsal_vectors))

In [536]:
dorsal_inv = np.linalg.inv(np.cov(pca_dorsal_vectors, rowvar=False))

In [537]:
mean = pca_dorsal_vectors.mean(axis=0)

In [538]:
diff = pca_dorsal_vectors - mean

In [539]:
diff.shape

(150, 20)

In [540]:
md = []
for i in range(len(diff)):
    md.append(
        np.sqrt(
            diff[i].dot(dorsal_inv).dot(diff[i])
        )
    )

In [545]:
std = np.std(md)
k = std
m = np.mean(md)
up_t = m+k
low_t = m-k
for i in range(len(md)):
    if (md[i] >= up_t) or (md[i] <= low_t):
        print(i)

2
4
5
6
7
11
13
17
18
23
25
31
32
39
42
43
47
49
66
76
90
96
114
138


In [546]:
diff.shape

(150, 20)

In [547]:
test_labels

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [491]:
dorsal_vectors.shape

(50, 6)